In [2]:
import torch
import pandas as pd
from datetime import timedelta
from datetime import datetime

import numpy as np

from nera.data import *
from nera.models.ratings import EloAnalytical, EloManual, EloNumerical, EloSymbolical
from nera.reference import *
from nera.trainer import Trainer
from nera.utils import print_rating_diff


/home/kubos/anaconda3/envs/bp/lib/python3.11/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
da = DataAcquisition()
df = da.get_data(FROM_CSV, fname="../resources/other_leagues.csv")
df['DT'] = pd.to_datetime(df['DT'], format="%Y-%m-%d %H:%M:%S")
#df = df[(df['League'] != 'EuroLeague') & (df['League'] != 'EuroCup')] 
df = df.reset_index()
df = df.sort_values(by='DT', ascending=False)

transform = DataTransformation(df, timedelta(365))
dataset = transform.get_dataset(node_f_extract=False, edge_f_one_hot=True)

team_count = transform.num_teams

print(len(df))

# dummy dataset

delta = timedelta(seconds=1)
delta2 = timedelta(days=366)
now = datetime.now()
data = pd.DataFrame({'DT': [*[now - i * delta2 + j * delta for j in range(3) for i in range(4)]], 
                     'Home': ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C'],
                     'Away': ['B', 'C', 'A', 'C', 'A', 'B', 'B', 'C', 'A', 'C', 'A', 'B'],
                     'Winner': ['home', 'away', 'away', 'home', 'away', 'away', 'home', 'home', 'away', 'home', 'away', 'home'],
                     'Home_points': [10, 5, 15, 12, 15, 6, 20, 10, 10, 14, 3, 12],
                     'Away_points': [4, 8, 17, 10, 16, 14, 18, 9, 15, 0, 11, 4],
                     'League': [*(12 * ['liga'])],
                     })
data = data.sort_values(by='DT', ascending=False)
transform2 = DataTransformation(data, timedelta(days=365))
dummy_dataset = transform2.get_dataset(edge_f_one_hot=True)

elo_man = EloManual(team_count=transform2.num_teams)
trainer = Trainer(dummy_dataset, train_ratio=1)
trainer.model = elo_man
reference_maker = RatingReference(transform2.num_teams)

2024-03-07 14:26:45.544 | INFO     | nera.data._data_saving_loading:load_data_csv:70 - 21100 rows loaded from ../resources/other_leagues.csv


21100


### Manual Elo with NN pass

In [4]:
print('Computed elo Dummy: ', reference_maker.compute_reference('elo', dummy_dataset)[0])
acc_dummy = trainer.train(epochs=1, val_ratio=0, verbose=True)
print(elo_man.elo)
print()
print('----------------------------------------')
print()

reference_maker.num_teams = transform.num_teams
computed = reference_maker.compute_reference('elo', dataset)[0]

print()
elo = EloManual(team_count=transform.num_teams)
trainer.dataset = dataset
trainer.model = elo
acc_late = trainer.train(epochs=1, val_ratio=0, verbose=True)

err = False
for i in range(len(computed)):
    cmp = float(computed[i])
    net = float(elo.elo[i])
    if abs(cmp - net) > 0.1:
        print(rf'ERROR on index {i}:: {cmp} / {net}')
        err = True
    if i < 5:
        print(f'{i}:: computed: {cmp:10.3f} || net: {net:10.3f}')
if not err:
    print("...")
    print("[SUCCESS]: Computed elo is the same as Elo from NN")



2024-03-07 14:16:28.600 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 174.165, training accuracy: 66.67%


Computed elo Dummy:  [ 658.38739014  880.46270752 1461.15014648]
Parameter containing:
tensor([ 658.3874,  880.4627, 1461.1500], dtype=torch.float64,
       requires_grad=True)

----------------------------------------


2024-03-07 14:16:37.171 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 2293848.884, training accuracy: 58.84%


0:: computed:   7191.367 || net:   7191.367
1:: computed:   2895.177 || net:   2895.178
2:: computed:   3006.513 || net:   3006.513
3:: computed:   4540.632 || net:   4540.633
4:: computed:   3065.280 || net:   3065.280
...
[SUCCESS]: Computed elo is the same as Elo from NN


### Elo with gradient

first lets check our analytical backward pass

In [12]:
from torch.autograd import gradcheck
from nera.models.ratings._elo._analytical import _elo_fn
# gradcheck takes a tuple of tensors as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.
input = (torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True))
test = gradcheck(_elo_fn, input, eps=1e-6, atol=1e-4)
print(test)

True


Define training function for elo

Now, let's train elo with analytical backward pass

In [14]:
_elo_params = {
    'gamma': 0.6,
    'c': 3.,
    'd': 500.,
}
trainer.set_lr(7, 'rating')
elo_grad = EloAnalytical(team_count, hp_grad=False, **_elo_params)
trainer.model = elo_grad
trainer.train_ratio = 0.8
trainer.train(verbose=True, epochs=1, val_ratio=0.2)
trainer.test(verbose=True)

2024-03-07 12:59:40.430 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 31544.302, training accuracy: 50.11%
2024-03-07 12:59:40.432 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 0, validation loss: 8022.393, validation accuracy: 50.19%
2024-03-07 12:59:40.435 | INFO     | nera.trainer:test:267 - [TST] Testing accuracy: 42.71%


0.4270833333333333

And Elo with autograd (numerical gradient backward pass)

In [15]:
elo_auto = EloNumerical(team_count, hp_grad=False, **_elo_params)
trainer.model = elo_auto
trainer.train(verbose=True, epochs=1, val_ratio=0.2)
trainer.test(verbose=True)

2024-03-07 12:59:49.306 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 31542.678, training accuracy: 50.40%
2024-03-07 12:59:49.308 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 0, validation loss: 8032.279, validation accuracy: 50.89%
2024-03-07 12:59:49.309 | INFO     | nera.trainer:test:267 - [TST] Testing accuracy: 46.88%


0.46875

And finally, compare wheter these two are the same
___________________

In [16]:
print_rating_diff(elo_auto, elo_grad, transform)

Ratings diff:
Rating 0:
[ERROR] in rating 0 on index 0:: 18767.727127396844 / -16572.66731473815
[ERROR] in rating 0 on index 1:: -17715.781470317506 / -10968.81145372411
[ERROR] in rating 0 on index 2:: -6512.368626225078 / -2739.371559912648
[ERROR] in rating 0 on index 3:: 21005.999473362 / -765.711830564877
[ERROR] in rating 0 on index 4:: 20387.303889517465 / 10654.086947696178
...

Total number of errors: 139 out of 219 computed ratings
Cumulative sum of errors: 1585352.142484664
Average difference: 11405.411097011971
Average difference percentage: 86.8208152784141%
Maximal difference: 46178.492229172836
-------------------------

Hyperparams diff:


Now let's see what happens, when we let the model learn the c, d metaparameters as well

In [17]:
trainer.set_lr(0.0001, 'hyperparam')
trainer.set_lr(25, 'rating')
epoch_count = 5

elo_auto2 = EloNumerical(team_count, hp_grad=True, c=torch.tensor(3, dtype=torch.float64), d=torch.tensor(500, dtype=torch.float64))
trainer.model = elo_auto2
trainer.train(verbose=True, epochs=epoch_count, val_ratio=0.2)
trainer.test(verbose=True)

2024-03-07 12:59:59.485 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 1780160.303, training accuracy: 50.17%
2024-03-07 12:59:59.486 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 0, validation loss: 475771.900, validation accuracy: 50.38%
2024-03-07 13:00:09.672 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 1, training loss: 1633383.901, training accuracy: 49.95%
2024-03-07 13:00:09.673 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 1, validation loss: 424018.200, validation accuracy: 49.62%
2024-03-07 13:00:20.141 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 2, training loss: 1165865.505, training accuracy: 49.87%
2024-03-07 13:00:20.143 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 2, validation loss: 303593.915, validation accuracy: 50.08%
2024-03-07 13:00:30.874 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 3, training loss: 982259.866, training accuracy: 50.23%
2024-03-07 13:00:30.875 | INFO     | nera.trainer:train:174 - [VAL] 

0.5645833333333333

In [18]:
elo_grad2 = EloAnalytical(team_count, hp_grad=True, c=torch.tensor(3, dtype=torch.float64), d=torch.tensor(500, dtype=torch.float64))
trainer.model = elo_grad2
trainer.train(verbose=True, epochs=epoch_count, val_ratio=0.2)
trainer.test(verbose=True)

2024-03-07 13:00:52.357 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 1778901.638, training accuracy: 49.77%
2024-03-07 13:00:52.359 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 0, validation loss: 472144.944, validation accuracy: 50.75%
2024-03-07 13:01:04.622 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 1, training loss: 1738533.913, training accuracy: 49.28%
2024-03-07 13:01:04.623 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 1, validation loss: 441891.658, validation accuracy: 47.99%
2024-03-07 13:01:16.544 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 2, training loss: 1587505.247, training accuracy: 50.47%
2024-03-07 13:01:16.546 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 2, validation loss: 406042.658, validation accuracy: 50.48%
2024-03-07 13:01:27.374 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 3, training loss: 998616.636, training accuracy: 51.07%
2024-03-07 13:01:27.375 | INFO     | nera.trainer:train:174 - [VAL] 

0.4979166666666667

In [19]:
print_rating_diff(elo_auto2, elo_grad2, transform)

Ratings diff:
Rating 0:
[ERROR] in rating 0 on index 0:: -8888.75005067641 / -10131.376043258608
[ERROR] in rating 0 on index 1:: -16313.612330261521 / 17446.453963609936
[ERROR] in rating 0 on index 2:: 9846.623066015953 / -3010.313957015063
[ERROR] in rating 0 on index 3:: -15524.33175620385 / 14175.394007912557
[ERROR] in rating 0 on index 4:: -5851.881058257431 / -19520.280396710572
...

Total number of errors: 201 out of 219 computed ratings
Cumulative sum of errors: 5057301.206906568
Average difference: 25160.702521923224
Average difference percentage: 141.85120883074623%
Maximal difference: 107718.35669714474
-------------------------

Hyperparams diff:
Numerical: hyperparam[0] =    0.978 :: Numerical: hyperparam[0] =    1.024
Numerical: hyperparam[1] =  506.215 :: Numerical: hyperparam[1] =  506.209


In [20]:
elo = EloManual(team_count=transform.num_teams)
trainer.dataset = dataset
trainer.model = elo
acc_late = trainer.train(epochs=1, val_ratio=0.2, verbose=True)
trainer.test(verbose=True)

2024-03-07 13:01:42.870 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 1620577.841, training accuracy: 59.20%
2024-03-07 13:01:42.871 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 0, validation loss: 424847.971, validation accuracy: 57.11%
2024-03-07 13:01:42.873 | INFO     | nera.trainer:test:267 - [TST] Testing accuracy: 56.25%


0.5625

### SYMBOLICAL ELO

In [9]:
elo_sym = EloSymbolical(team_count=transform.num_teams, rating_dim=3, hp_grad=True)
trainer.dataset = dataset
trainer.model = elo_sym
trainer.train_ratio = 0.8
trainer.set_lr(0.02, 'rating')
acc_sym = trainer.train(epochs=5, val_ratio=0.2, verbose=True)
trainer.test(verbose=True)

2024-03-07 15:57:54.174 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: nan, training accuracy: 0.00%
2024-03-07 15:57:54.175 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 0, validation loss: nan, validation accuracy: 0.00%
2024-03-07 15:58:05.635 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 1, training loss: nan, training accuracy: 0.00%
2024-03-07 15:58:05.636 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 1, validation loss: nan, validation accuracy: 0.00%
2024-03-07 15:58:17.044 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 2, training loss: nan, training accuracy: 0.00%
2024-03-07 15:58:17.045 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 2, validation loss: nan, validation accuracy: 0.00%
2024-03-07 15:58:29.097 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 3, training loss: nan, training accuracy: 0.00%
2024-03-07 15:58:29.098 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 3, validation loss: nan, validation accuracy: 0.00%


0.0

In [ ]:
print(elo_sym.elo[:3])